<a href="https://colab.research.google.com/github/kluo9/Deap-Learning/blob/main/Speaker_identification_with_VoxCeleb_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>